### 확률적 경사 하강법(SGD)을 이용한 행렬 분해

In [2]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정. 
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

num_users, num_items = R.shape #(4,5)
K=3

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다. 
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))
print("P:",P)
print("Q:",Q)

P: [[ 0.54144845 -0.2039188  -0.17605725]
 [-0.35765621  0.28846921 -0.76717957]
 [ 0.58160392 -0.25373563  0.10634637]
 [-0.08312346  0.48736931 -0.68671357]]
Q: [[-0.1074724  -0.12801812  0.37792315]
 [-0.36663042 -0.05747607 -0.29261947]
 [ 0.01407125  0.19427174 -0.36687306]
 [ 0.38157457  0.30053024  0.16749811]
 [ 0.30028532 -0.22790929 -0.04096341]]


##### scale 파라미터를 1/K 로 설정한 이유는 무엇인가에 대한 저자의 답변
본 예제에서는 random 정규분포의 표준편차가 작지 않으면 무한대로 수렴하는 경우가 발생할 수 있어서 1/K를 적용했습니다.

In [3]:
# 실제 R 행렬과 예측 행렬의 오차를 구하는 get_rmse() 함수
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    full_pred_matrix = np.dot(P, Q.T) # 예측 R 행렬
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측행렬의 RMSE 추출
    x_non_zero_idx = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_idx = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_idx, y_non_zero_idx]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_idx, y_non_zero_idx]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse    

In [4]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.48767231013696477
### iteration step :  100  rmse :  0.1564340384819248
### iteration step :  150  rmse :  0.07455141311978032
### iteration step :  200  rmse :  0.0432522679857931
### iteration step :  250  rmse :  0.029248328780879088
### iteration step :  300  rmse :  0.022621116143829344
### iteration step :  350  rmse :  0.01949363619652533
### iteration step :  400  rmse :  0.018022719092132503
### iteration step :  450  rmse :  0.01731968595344277
### iteration step :  500  rmse :  0.01697365788757103
### iteration step :  550  rmse :  0.016796804595895533
### iteration step :  600  rmse :  0.01670132290188455
### iteration step :  650  rmse :  0.016644736912476654
### iteration step :  700  rmse :  0.016605910068210192
### iteration step :  750  rmse :  0.01657420047570466
### iteration step :  800  rmse :  0.016544315829215932
### iteration step :  850  rmse :  0.01651375177473506
### itera

In [6]:
# non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

# non_zeros=[]
# for j in range(num_items):
#     for i in range(num_users):
#         if R[i,j]>0:
#             non_zeros.append(i,j,R[i,j])


In [5]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


In [7]:
# 원본 행렬
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
R

array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])